In [1]:
!pip install yahoo_fin

In [2]:
!pip install requests_html

**We found the below method of downloading Yahoo Finance information from the statistics page on the internet and it seems to work. Alternativly, one could go into each of the ticker codes of Yahoo Finance page**


In [3]:
import yahoo_fin.stock_info as si #Get data
import pandas as pd #Data manipulation
from tqdm import tqdm #Make a progress bar because that's cool...
from bs4 import BeautifulSoup # to parse external data
import requests # to get data
from requests_html import HTMLSession
import matplotlib.pyplot as plt
import plotly.express as px
import pickle

In [4]:
def getEURSTX50tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/EURO_STOXX_50')
    soup = BeautifulSoup(resp.text, 'lxml')
    tableEURSTX = soup.find(text="Ticker").find_parent("table").find('tbody').findAll('tr')[1:]

    EURSTX_tickers = []
    for row in tableEURSTX:
        ticker1 = row.findAll('td')[0].text.strip()
        EURSTX_tickers.append(ticker1)
    with open("EURSTX50tickers.pickle", "wb") as f:
        pickle.dump(EURSTX_tickers, f)
    return EURSTX_tickers

ER_tickers = getEURSTX50tickers()
ER_tickers

['ADS.DE',
 'ADYEN.AS',
 'AD.AS',
 'AI.PA',
 'AIR.PA',
 'ALV.DE',
 'ABI.BR',
 'ASML.AS',
 'CS.PA',
 'BAS.DE',
 'BAYN.DE',
 'BBVA.MC',
 'SAN.MC',
 'BMW.DE',
 'BNP.PA',
 'CRG.IR',
 'DAI.DE',
 'BN.PA',
 'DB1.DE',
 'DPW.DE',
 'DTE.DE',
 'ENEL.MI',
 'ENI.MI',
 'EL.PA',
 'FLTR.IR',
 'IBE.MC',
 'ITX.MC',
 'IFX.DE',
 'INGA.AS',
 'ISP.MI',
 'KER.PA',
 'KNEBV.HE',
 'OR.PA',
 'LIN.DE',
 'MC.PA',
 'MUV2.DE',
 'RI.PA',
 'PHIA.AS',
 'PRX.AS',
 'SAF.PA',
 'SAN.PA',
 'SAP.DE',
 'SU.PA',
 'SIE.DE',
 'STLA.MI',
 'TTE.PA',
 'UMG.AS',
 'DG.PA',
 'VOW.DE',
 'VNA.DE']

In [5]:
def valuation_measures(reload_EURSTX50=False):
       
    if reload_EURSTX50:
        EURSTX_tickers = getEURSTX50tickers()
    else:
        with open("EURSTX50tickers.pickle","rb") as f:
            EURSTX_tickers = pickle.load(f)
    ticker_stats = {}     
    for ticker in EURSTX_tickers:
        try:
            df = si.get_stats_valuation(ticker)
            df = df.iloc[:,:2]
            df.columns = ["Attribute", "Recent"]
            ticker_stats[ticker] = df
        except:
            pass
    dat = pd.concat(ticker_stats)
    dat = dat.reset_index()
    dat = dat.dropna()
    del dat["level_1"]
    dat.columns = ["Ticker", "Attribute", "Recent"]
    dat.to_csv('df1.csv')
    return dat
    

In [6]:
df1 = valuation_measures()
df1


,Ticker,Attribute,Recent


In [7]:
def extra_stats(reload_EURSTX50=False):
    
    if reload_EURSTX50:
        EURSTX_tickers = getEURSTX50tickers()
    else:
        with open("EURSTX50tickers.pickle","rb") as f:
            EURSTX_tickers = pickle.load(f)
    ticker_extra_stats = {}
    for ticker in tqdm(ER_tickers):
        try:
            ticker_extra_stats[ticker] = si.get_stats(ticker)
        except:
            pass
    dat2 = pd.concat(ticker_extra_stats)
    dat2 = dat2.reset_index()
    dat2 = dat2.dropna()
    del dat2["level_1"]
    dat2.columns = ["Ticker", "Attribute", "Value"]
    dat2.to_csv('df2.csv')
    return dat2

In [8]:
df2 = extra_stats()
df2

100%|██████████| 50/50 [02:42<00:00,  3.26s/it]


,Ticker,Attribute,Value
0,ADS.DE,Beta (5Y Monthly),0.89
3,ADS.DE,52 Week High 3,336.25
4,ADS.DE,52 Week Low 3,231.55
5,ADS.DE,50-Day Moving Average 3,251.12
6,ADS.DE,200-Day Moving Average 3,282.58
...,...,...,...
2314,VNA.DE,Total Debt/Equity (mrq),162.57
2315,VNA.DE,Current Ratio (mrq),0.39
2316,VNA.DE,Book Value Per Share (mrq),47.85
2317,VNA.DE,Operating Cash Flow (ttm),1.44B
